In [1]:
# IP USED THROUGHOUT THE ASSIGNMENT
hostip = "172.22.32.1"

In [ ]:
import time
from datetime import datetime
from pymongo import MongoClient
from kafka3 import KafkaProducer
import json
from bson.json_util import dumps
from kafka3.admin import KafkaAdminClient, NewTopic
import time

def recreate_topics(bootstrap_servers:str, topics:str,
                    num_partitions:int=1, replication_factor:int=1):
    """
    This function are created to help purge Kafka producer streams we just created to
    
    """
    # Ensure we pass a list of host:port strings
    if isinstance(bootstrap_servers, str):
        bs_list = [bootstrap_servers]
    else:
        bs_list = bootstrap_servers

    admin = KafkaAdminClient(bootstrap_servers=bs_list)
    try:
        admin.delete_topics(topics)
        print(f"[INFO] Deletion of topics {topics} initiated.")
    except Exception as e:
        print(f"[WARN] Could not delete topics: {e}")
    finally:
        admin.close()


# Kafka setup
topic = "mongo_plot_stream"
kafka_server = f"{hostip}:9092"

# PURGE & RECREATE before producing
recreate_topics(kafka_server, [topic],
                num_partitions=1,
                replication_factor=1)

# ── Setup ──
mongo_client = MongoClient(f"mongodb://{hostip}:27017")
db           = mongo_client["fit3182_db"]
collection   = db["Violation"]

producer = KafkaProducer(
    bootstrap_servers=kafka_server,            # your Kafka broker
    value_serializer=lambda v: dumps(v).encode('utf-8')
)
stream_topic = topic

# ── Initialize cursor at 2024-01-01T00:00:00Z ──
last_ts = datetime(2024, 1, 1)

try:
    while True:
        # find any document with at least one new violation
        query = {"violations.timestamp_start": {"$gt": last_ts}}
        # sort by the embedded timestamp so you process older docs first
        new_docs = list(collection
                        .find(query)
                        .sort("violations.timestamp_start", 1))

        for doc in new_docs:
            # send the full document, preserving original structure
            producer.send(stream_topic, doc)
            print(f"✅ Sent to Kafka ({stream_topic}): {doc}")

            # bump last_ts to the newest violation in this doc
            timestamps = [v["timestamp_start"] for v in doc["violations"]]
            max_ts = min(timestamps)
            if max_ts > last_ts:
                last_ts = max_ts

        producer.flush()
        time.sleep(3)

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    producer.close()

[WARN] Could not delete topics: [Error 3] UnknownTopicOrPartitionError: Request 'DeleteTopicsRequest_v3(topics=['mongo_plot_stream'], timeout=30000)' failed with response 'DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='mongo_plot_stream', error_code=3)])'.
✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837061152939273f1dca2cc'), 'car_plate': 'KKE 15', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': '1621c4aa-756a-47c4-96ad-52af457b9a7b', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 0), 'timestamp_end': None, 'measured_speed': 150.3}, {'violation_id': '559a25fb-2d40-436a-86ec-3a00b16e0099', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 0, 23, 896000), 'timestamp_end': None, 'measured_speed': 138.3}, {'violation_id': '20015874-c46e-4e71-b0d3-4d1f5096e696', 'type': 'average', '

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837066c52939273f1dca3bd'), 'car_plate': 'AFS 0', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': '90834ab1-6fa1-4e72-8b52-23129e87c60c', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 25, 20), 'timestamp_end': None, 'measured_speed': 134.1}, {'violation_id': 'baa229db-af9b-48cb-b322-eae8695693b0', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 25, 46, 40000), 'timestamp_end': None, 'measured_speed': 126.8}, {'violation_id': 'dabfabec-5559-4fc1-b2f7-113ee3717859', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 25, 20), 'timestamp_end': datetime.datetime(2024, 1, 1, 8, 25, 46, 40000), 'measured_speed': 130.45}, {'violation_id': '06ce66a7-a1a2-4617-857b-d9fb5dbc8637', 'type': 'instantaneous', 

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837066c52939273f1dca3a7'), 'car_plate': 'SAL 1597', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': '3c51d865-1249-4655-be58-03394510caf0', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': None, 'measured_speed': 141.1}, {'violation_id': 'ebe96d47-c481-4f14-b424-b24845300b1f', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'timestamp_end': None, 'measured_speed': 139.1}, {'violation_id': 'b77e6483-18d1-4131-adbc-01c22cb29a4d', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'measured_speed': 140.1}, {'violation_id': '9bee8a84-6914-41ca-b61f-920728f053e3', 'type': 'instantaneou

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837066c52939273f1dca3a7'), 'car_plate': 'SAL 1597', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': '3c51d865-1249-4655-be58-03394510caf0', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': None, 'measured_speed': 141.1}, {'violation_id': 'ebe96d47-c481-4f14-b424-b24845300b1f', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'timestamp_end': None, 'measured_speed': 139.1}, {'violation_id': 'b77e6483-18d1-4131-adbc-01c22cb29a4d', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'measured_speed': 140.1}, {'violation_id': '9bee8a84-6914-41ca-b61f-920728f053e3', 'type': 'instantaneou

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837066c52939273f1dca3a7'), 'car_plate': 'SAL 1597', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': '3c51d865-1249-4655-be58-03394510caf0', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': None, 'measured_speed': 141.1}, {'violation_id': 'ebe96d47-c481-4f14-b424-b24845300b1f', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'timestamp_end': None, 'measured_speed': 139.1}, {'violation_id': 'b77e6483-18d1-4131-adbc-01c22cb29a4d', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 8, 35, 23), 'timestamp_end': datetime.datetime(2024, 1, 1, 8, 35, 49, 319000), 'measured_speed': 140.1}, {'violation_id': '9bee8a84-6914-41ca-b61f-920728f053e3', 'type': 'instantaneou

...

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837088152939273f1dca7d3'), 'car_plate': 'YV 2238', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': 'a68b50f4-459c-4961-a7e6-b73e79c31429', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': None, 'measured_speed': 149.6}, {'violation_id': '45b01527-c9c8-407e-b177-dcec92b1d2f4', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'timestamp_end': None, 'measured_speed': 151.4}, {'violation_id': 'd2d8d44e-7e7f-4e22-ba19-2feb700345cd', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'measured_speed': 150.5}, {'violation_id': '9991500b-4e79-4ed9-a355-cd68caf6a46d', 'type': 'instantaneous',

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837088152939273f1dca7d3'), 'car_plate': 'YV 2238', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': 'a68b50f4-459c-4961-a7e6-b73e79c31429', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': None, 'measured_speed': 149.6}, {'violation_id': '45b01527-c9c8-407e-b177-dcec92b1d2f4', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'timestamp_end': None, 'measured_speed': 151.4}, {'violation_id': 'd2d8d44e-7e7f-4e22-ba19-2feb700345cd', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'measured_speed': 150.5}, {'violation_id': '9991500b-4e79-4ed9-a355-cd68caf6a46d', 'type': 'instantaneous',

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837088152939273f1dca7d3'), 'car_plate': 'YV 2238', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': 'a68b50f4-459c-4961-a7e6-b73e79c31429', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': None, 'measured_speed': 149.6}, {'violation_id': '45b01527-c9c8-407e-b177-dcec92b1d2f4', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'timestamp_end': None, 'measured_speed': 151.4}, {'violation_id': 'd2d8d44e-7e7f-4e22-ba19-2feb700345cd', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'measured_speed': 150.5}, {'violation_id': '9991500b-4e79-4ed9-a355-cd68caf6a46d', 'type': 'instantaneous',

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837088152939273f1dca7d3'), 'car_plate': 'YV 2238', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': 'a68b50f4-459c-4961-a7e6-b73e79c31429', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': None, 'measured_speed': 149.6}, {'violation_id': '45b01527-c9c8-407e-b177-dcec92b1d2f4', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'timestamp_end': None, 'measured_speed': 151.4}, {'violation_id': 'd2d8d44e-7e7f-4e22-ba19-2feb700345cd', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'measured_speed': 150.5}, {'violation_id': '9991500b-4e79-4ed9-a355-cd68caf6a46d', 'type': 'instantaneous',

✅ Sent to Kafka (mongo_plot_stream): {'_id': ObjectId('6837088152939273f1dca7d3'), 'car_plate': 'YV 2238', 'date': datetime.datetime(2024, 1, 1, 0, 0), 'violations': [{'violation_id': 'a68b50f4-459c-4961-a7e6-b73e79c31429', 'type': 'instantaneous', 'camera_id_start': 1, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': None, 'measured_speed': 149.6}, {'violation_id': '45b01527-c9c8-407e-b177-dcec92b1d2f4', 'type': 'instantaneous', 'camera_id_start': 2, 'camera_id_end': None, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'timestamp_end': None, 'measured_speed': 151.4}, {'violation_id': 'd2d8d44e-7e7f-4e22-ba19-2feb700345cd', 'type': 'average', 'camera_id_start': 1, 'camera_id_end': 2, 'timestamp_start': datetime.datetime(2024, 1, 1, 9, 24, 45), 'timestamp_end': datetime.datetime(2024, 1, 1, 9, 25, 8, 304000), 'measured_speed': 150.5}, {'violation_id': '9991500b-4e79-4ed9-a355-cd68caf6a46d', 'type': 'instantaneous',